# item_cf

In [78]:
!python baseline0_itemCF.py

gen train_data_matrix begin...
cal cosine_similarity begin...
cal cosine_similarity end
100%|████████████████████████████████████| 27195/27195 [00:45<00:00, 600.60it/s]
935.4970018863678s
complete!!!


In [79]:
import time
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)  # 设置显示数据的最大列数，防止出现省略号…，导致数据显示不全
pd.set_option('expand_frame_repr', False)  # 当列太多时不自动换行

t0 = time.time()

path = './data_origin/'

train_click_0_df = pd.read_csv(path+'underexpose_train/underexpose_train_click-0.csv',names=['user_id','item_id','time'])
train_click_1_df = pd.read_csv(path+'underexpose_train/underexpose_train_click-1.csv',names=['user_id','item_id','time'])
train_click_df = pd.concat([train_click_0_df, train_click_1_df], ignore_index=True)

test_click_0_df = pd.read_csv(path+'underexpose_test/underexpose_test_click-0/underexpose_test_click-0.csv', names=['user_id','item_id','time'])
test_click_1_df = pd.read_csv(path+'underexpose_test/underexpose_test_click-1/underexpose_test_click-1.csv', names=['user_id','item_id','time'])

test_click_df = pd.concat([test_click_0_df, test_click_1_df], ignore_index=True)

click_df = pd.concat([train_click_df, test_click_df], ignore_index=True)

In [80]:
click_df.groupby('user_id')['item_id'].count().describe()

count    21457.000000
mean        24.681782
std         23.714898
min          2.000000
25%          9.000000
50%         17.000000
75%         32.000000
max        293.000000
Name: item_id, dtype: float64